In [ ]:
import dask.dataframe as dd
import matplotlib.pyplot as plt
import os
from sklearn.metrics import r2_score
import dask_ml
import dask
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*Sending large graph.*")

from dask.distributed import Client, LocalCluster
import dask.multiprocessing

cluster = LocalCluster(processes=True,n_workers=6, threads_per_worker=1)
client = Client(cluster)
import sys

In [ ]:
folders = [
    'train0_25',
    'train25_50',
    'train50_75',
    'train75_100'
]

# Read Parquet files from each folder into Dask DataFrames
dfs = [dd.read_parquet(folder) for folder in folders]

# Concatenate all DataFrames into a single DataFrame
data = dd.concat(dfs)

In [ ]:
feat60 = ['state_t', 'state_q0001','state_q0002','state_q0003','state_u','state_v','pbuf_ozone','pbuf_CH4','pbuf_N2O']
feat1 = ['state_ps','pbuf_SOLIN','pbuf_LHFLX','pbuf_SHFLX','pbuf_TAUX','pbuf_TAUY','pbuf_COSZRS','cam_in_ALDIF','cam_in_ALDIR','cam_in_ASDIF','cam_in_ASDIR','cam_in_LWUP','cam_in_ICEFRAC','cam_in_LANDFRAC','cam_in_OCNFRAC','cam_in_SNOWHLAND']

target60 = ['ptend_t','ptend_q0001','ptend_q0002','ptend_q0003','ptend_u','ptend_v']
target1 = ['cam_out_NETSW','cam_out_FLWDS','cam_out_PRECSC','cam_out_PRECC','cam_out_SOLS','cam_out_SOLL','cam_out_SOLSD','cam_out_SOLLD']

features60 = [] 
for f in feat60:
    features60 = features60 + [f+'_'+str(i) for i in range(60)]
allF = features60 + feat1

targets60 = [] 
for f in target60:
    targets60 = targets60 + [f+'_'+str(i) for i in range(60)]
allT = targets60 + target1

targetsToDrop12 = [ 'ptend_q0001', 'ptend_q0002', 'ptend_q0003', 'ptend_u', 'ptend_v']
dropT = ['ptend_q0002_12','ptend_q0002_13','ptend_q0002_14'] # attention, I think i also need to predict _15
for f in targetsToDrop12:
    dropT = dropT + [f+'_'+str(i) for i in range(12)]

allT2 = [i for i in allT if i not in dropT]

In [ ]:
np.random.seed(42)

orig_partitions = [i for i in range(0,int(data.npartitions))]
np.random.shuffle(orig_partitions) #shuffles inplace

trainSep = int(0.95* data.npartitions)
valEnd = data.npartitions #int(0.05* data.npartitions) + trainSep

sampledPartIdxTrain = orig_partitions[0:trainSep]
sampledPartIdxTest  = orig_partitions[trainSep:valEnd]

In [ ]:
n60Feat = len(feat60)
n1dFeat = len(feat1)
n60Targ = len(target60)
n1dTarg = len(target1)

# data processing

In [ ]:
import tensorflow as tf

from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM, Embedding, Concatenate,BatchNormalization, Reshape
from tensorflow.keras.models import Model

In [ ]:
from keras import backend as K

def r2_scoretf(y_true, y_pred):
    sum_squares_residuals = tf.reduce_sum(tf.square(y_true - y_pred), axis=0)
    sum_squares_total = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)), axis=0)
    r2 = 1 - (sum_squares_residuals / sum_squares_total)
    return r2 #tf.reduce_mean(r2)

def r2_scoreTrain(y_true, y_pred):
    sum_squares_residuals = tf.reduce_sum(tf.square(y_true - y_pred), axis=0)
    sum_squares_total = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)), axis=0)
    r2 = (sum_squares_residuals / sum_squares_total) # alwaysPositive, the smaller the better
    return tf.reduce_mean(r2)

class RSquaredMetric(tf.keras.metrics.Metric):
    def __init__(self, name='r_squared', **kwargs):
        super().__init__(name=name, **kwargs)
        self.total_sum_squares = None#self.add_weight(name='total_sum_squares', initializer='zeros', shape=shape)
        self.residual_sum_squares = None#self.add_weight(name='residual_sum_squares', initializer='zeros', shape=shape)
        self.num_samples = self.add_weight(name="num_samples", initializer='zeros',dtype=tf.int32)
 
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, self._dtype)
        y_pred = tf.cast(y_pred, self._dtype)
        
        sum_squares_residuals = tf.reduce_sum(tf.square(y_true - y_pred), axis=0)
        sum_squares_total = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true, axis=0)), axis=0)
        sum_squares_total = tf.where(tf.equal(sum_squares_total, 0.0), tf.ones_like(sum_squares_total), sum_squares_total)
        
        if self.total_sum_squares is None:
            self.total_sum_squares = self.add_weight(name='total_sum_squares', initializer='zeros', shape=sum_squares_total.shape)
            self.residual_sum_squares = self.add_weight(name='residual_sum_squares', initializer='zeros', shape=sum_squares_residuals.shape)

        self.total_sum_squares.assign_add(sum_squares_total)
        self.residual_sum_squares.assign_add(sum_squares_residuals)

    def result(self):
        r_squared = 1 - (self.residual_sum_squares / self.total_sum_squares)
        r_squared = tf.where(tf.math.is_nan(r_squared), tf.ones_like(r_squared), r_squared)
        return tf.reduce_mean(r_squared)

    def reset_state(self):
        self.total_sum_squares.assign(tf.zeros_like(self.total_sum_squares))
        self.residual_sum_squares.assign(tf.zeros_like(self.residual_sum_squares))

In [ ]:
m = RSquaredMetric((60,n60Targ))
m.update_state(y2d_val, y2d_pred)
m.result()

In [ ]:
m = RSquaredMetric(n1dTarg)
m.update_state(y1d_val, y1d_val)
m.result()

In [ ]:
def addFeatures(a):
    addF = []
    colDict={}
    for i in range(60):
        newF = f'absWind_{i}'
        addF.append(newF)
        colDict[newF] =(a[f'state_u_{i}']**2 + a[f'state_v_{i}']**2)

        newF = f'angleWind_{i}'
        addF.append(newF)
        colDict[newF] =np.arctan2(a[f'state_u_{i}']**2,a[f'state_v_{i}']**2)
    a = pd.concat([a.reset_index(), pd.DataFrame(colDict).reset_index()], axis=1)

    colDict = {}
    for f in (feat60+['absWind']):
        for i in range(59):
            newF = 'diff'+f+'_'+str(i)
            addF.append(newF)
            colDict[newF] = (a[f+'_'+str(i+1)] - a[f+'_'+str(i)])
    a = pd.concat([a.reset_index(), pd.DataFrame(colDict).reset_index()], axis=1)
    return a, addF

def getTensorDataSeq(data, partPerLoop, startPartIdx,sampledPartIdx):
    X1d, X2d, y1d, y2d, X1dI, X2dI, y1dI,y2dI  = None, None, None, None, False, False, False, False
    for j in range(partPerLoop):
        a = data.get_partition(int(sampledPartIdx[startPartIdx+j])).compute()
        #a, newF = addFeatures(a)
        b = np.reshape(a[features60], (a.shape[0], n60Feat, 60))
        b = np.transpose(b, (0,2,1))
        X2d = np.concatenate([X2d,b], axis=0) if X2dI else b
        b = np.reshape(a[targets60], (a.shape[0], n60Targ, 60))
        b = np.transpose(b, (0,2,1))
        y2d = np.concatenate([y2d,b], axis=0) if y2dI else b
        X1d = np.concatenate([X1d,a[feat1]], axis=0) if X1dI else a[feat1]
        y1d = np.concatenate([y1d,a[target1]], axis=0) if y1dI else a[target1]
        X1dI, X2dI, y1dI,y2dI = True, True, True, True
    return X1d, X2d, y1d, y2d

def getTensorDataFlattend(data, partPerLoop, startPartIdx,sampledPartIdx):
    X, y, xi, yi = None, None, False, False
    for j in range(partPerLoop):
        a = data.get_partition(int(sampledPartIdx[startPartIdx+j])).compute()
        a, newF = addFeatures(a)
        allF = features60+newF+feat1
        b = a[allF]
        X = np.concatenate([X,b], axis=0) if xi else b
        
        b = a[targets60+target1]
        y = np.concatenate([y,b], axis=0) if yi else b

        xi, yi = True, True
    return X,y, allF

In [ ]:
# validation data
partPerLoop = 35

for i in range(1):
    startPartIdx = i*partPerLoop
    X_val, y_val, allF = getTensorDataFlattend(data, partPerLoop, startPartIdx, sampledPartIdxTest)

In [ ]:
# training sequentially
partPerLoop = 50

for i in range(1):
    startPartIdx = i*partPerLoop
    X,y, allF = getTensorDataFlattend(data, partPerLoop, startPartIdx, sampledPartIdxTrain)  

In [ ]:
allTargets = targets60+target1

In [ ]:
feat60 = ['state_t', 'state_q0001','state_q0002','state_q0003','state_u','state_v','pbuf_ozone','pbuf_CH4','pbuf_N2O']
feat1 = ['state_ps','pbuf_SOLIN','pbuf_LHFLX','pbuf_SHFLX','pbuf_TAUX','pbuf_TAUY','pbuf_COSZRS','cam_in_ALDIF','cam_in_ALDIR','cam_in_ASDIF','cam_in_ASDIR','cam_in_LWUP','cam_in_ICEFRAC','cam_in_LANDFRAC','cam_in_OCNFRAC','cam_in_SNOWHLAND']

target60 = ['ptend_t','ptend_q0001','ptend_q0002','ptend_q0003','ptend_u','ptend_v']
target1 = ['cam_out_NETSW','cam_out_FLWDS','cam_out_PRECSC','cam_out_PRECC','cam_out_SOLS','cam_out_SOLL','cam_out_SOLSD','cam_out_SOLLD']

a = data.get_partition(int(sampledPartIdxTrain[0])).compute()

bytes = sys.getsizeof(a)
print(bytes/1e6)

addF = []
colDict={}
for i in range(60):
    newF = f'absWind_{i}'
    addF.append(newF)
    colDict[newF] =(a[f'state_u_{i}']**2 + a[f'state_v_{i}']**2)
a = pd.concat([a.reset_index(), pd.DataFrame(colDict).reset_index()], axis=1)

colDict = {}
for f in (feat60+['absWind']):
    for i in range(59):
        newF = 'diff'+f+'_'+str(i)
        addF.append(newF)
        colDict[newF] = (a[f+'_'+str(i+1)] - a[f+'_'+str(i)])
a = pd.concat([a.reset_index(), pd.DataFrame(colDict).reset_index()], axis=1)

bytes = sys.getsizeof(a)
print(bytes/1e6)
#a = a.assign(**{f'diff'+f+'_'+str(i): (a[f+'_'+str(i+1)] - a[f+'_'+str(i)]) for i in range(59) for f in (feat60)}) #feat60+['absWind']
#
#a = a.assign(
#    **{f'absWind_{i}': (a[f'state_u_{i}']**2 + a[f'state_v_{i}']**2) for i in range(60)}
#)

#a = a.assign(**{f'diff'+f+'_'+str(i): (a[f+'_'+str(i+1)] - a[f+'_'+str(i)]) for i in range(59) for f in (['absWind'])}) #feat60+['absWind']



In [ ]:
mean = np.mean(y, axis=0)
std = np.std(y, axis=0)
std[std==0] = 1

yn = (y - mean) / std
yn_val = (y_val - mean) / std

In [ ]:
## shuffle dataset
np.random.seed(42)
permutation = np.random.permutation(X1d.shape[0])
X1d = X1d[permutation]
y1d = y1d[permutation]
X2d = X2d[permutation]
y2d = y2d[permutation]

# simple fully connected model
- doesn't work at all so far

In [ ]:
tf.random.set_seed(42)

OneDInput = Input(shape=(n1dFeat,))
TwoDInput = Input(shape=(60,n60Feat))

x = BatchNormalization()(TwoDInput)
x = Dense(n60Feat, activation='relu')(x)
print(x.shape)
for i in range(6):
    x = Dense(n60Feat, activation='relu')(x)
    print(x.shape)

# add info to 1d output
x0 = Dense(1, activation='relu')(x) #reduce to 1d
x0 = x0[:,:,0]
y = BatchNormalization()(OneDInput)
y = Dense(n1dFeat, activation='relu')(y)

y = Concatenate(axis=1)([x0, y])
print(y.shape)
for i in range(6):
    y = Dense(60+n1dFeat, activation='relu')(y)
    print(y.shape)
y = Dense(n1dTarg, activation='linear', name='1d')(y)



x = Dense(n60Targ, activation='linear',name='2d')(x)
print(x.shape)
output2d =x
output1d =y

model = Model(inputs=[TwoDInput, OneDInput], outputs=[output2d, output1d])
model.compile(optimizer='adam', loss='mse', metrics=[r2_score])

hist = model.fit([X2d, X1d], [y2d, y1d], epochs=10, batch_size=32, validation_data=([X2d_val, X1d_val],[y2d_val, y1d_val]))

In [ ]:
[y2d_pred, y1d_pred] = model.predict([X2d_val, X1d_val])

y2d_pred = np.reshape(y2d_pred, (y2d_pred.shape[0],-1))
y2d_val0 = np.reshape(y2d_val, (y2d_val.shape[0],-1))
r2_scores = []
f = np.reshape(np.reshape(np.array(targets60), (n60Targ,60)).transpose(), (1,-1))
for i in range(y2d_val0.shape[1]):
    r2 = r2_score(y2d_val0[:, i], y2d_pred[:, i])
    print(f[0][i], r2)
    r2_scores.append(r2)

r2_scores1d = []
for i in range(y1d_pred.shape[1]):
    r2 = r2_score(y1d_val[:, i], y1d_pred[:, i])
    print(target1[i], r2)
    r2_scores1d.append(r2)


In [ ]:
print('mean 2d',np.mean(np.array(r2_scores)))
print('mean 1d',np.mean(np.array(r2_scores1d)))

#oss: 2140.1865 - 2d_loss: 1.4499e-05 - 1d_loss: 2140.1865 - 2d_r2_score: -28348.7305 - 1d_r2_score: 0.9105 - val_loss: 2108.1790 - val_2d_loss: 1.6521e-06 - val_1d_loss: 2108.1790 - val_2d_r2_score: -3997.2546 - val_1d_r2_score: 0.9232

In [ ]:
a = pd.DataFrame(y1d_pred, columns=target1)
b = pd.DataFrame(y1d_val, columns=target1)

In [ ]:
a.cam_out_PRECSC.plot()
b.cam_out_PRECSC.plot()

# network for 1d and 2d
1d:
- snow & rain rate have big problems

## 1d

In [ ]:
tf.random.set_seed(42)

OneDInput = Input(shape=(n1dFeat,))
TwoDInput = Input(shape=(60,n60Feat), name='input')
x = Reshape((60 * n60Feat,),name='inputReshape')(TwoDInput)

x = BatchNormalization()(x)

x = Dense(60*n60Feat, activation='relu')(x)
print(x.shape)
for i in range(2):
    x = Dense((60/(1))*n60Feat, activation='relu')(x)
    print(x.shape)

# add info to 1d output
y = BatchNormalization()(OneDInput)
y = Dense(n1dFeat, activation='relu')(y)

commonLayer = Concatenate(axis=1)([x, y])
commonLayerSize = 60*n60Targ+n1dFeat
y = Dense(commonLayerSize, activation='relu')(commonLayer)
print('y',y.shape)
for i in range(2):
    y = Dense(int(commonLayerSize / (2*(i+1))), activation='relu')(y)
    print('y',y.shape)
y = Dense(n1dTarg, activation='relu', name='1d')(y)
print('y',y.shape)
output1d =y

x = Dense(commonLayerSize, activation='relu')(commonLayer)
x = Dense(60*n60Targ, activation='linear')(x)
print(x.shape)
output2d = Reshape((60, n60Targ),name='2d')(x)
print(output2d.shape)


model1 = Model(inputs=[TwoDInput, OneDInput], outputs=[output1d])
model1.compile(optimizer='adam', loss='mse', metrics=[RSquaredMetric(n1dTarg)])

hist = model1.fit([X2d, X1d], y1d, epochs=10, batch_size=32, validation_data=([X2d_val, X1d_val],y1d_val))

In [ ]:
y1d_pred = model1.predict([X2d_val, X1d_val])

r2_scores1d = []
for i in range(y1d_pred.shape[1]):
    r2 = r2_score(y1d_val[:, i], y1d_pred[:, i])
    print(target1[i], r2)
    r2_scores1d.append(r2)
print('mean 1d',np.mean(np.array(r2_scores1d)))

## 2d

In [ ]:
def squared_error_sum(y_true, y_pred):
    squared_error = tf.square(y_true - y_pred)
    return tf.reduce_sum(squared_error)

def r2error_mean(y_true, y_pred):
    sum_squares_residuals = tf.reduce_sum(tf.square(y_true - y_pred), axis=0)
    sum_squares_total = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true, axis=0)), axis=0)
    sum_squares_total = tf.where(tf.equal(sum_squares_total, 0.0), tf.ones_like(sum_squares_total), sum_squares_total)
    error = (sum_squares_residuals / sum_squares_total)
    error = tf.where(tf.math.is_nan(error), tf.ones_like(error), error)
    return tf.reduce_mean(error)

In [ ]:
tf.random.set_seed(42)

OneDInput = Input(shape=(n1dFeat,))
TwoDInput = Input(shape=(60,n60Feat), name='input')
x = Reshape((60 * n60Feat,),name='inputReshape')(TwoDInput)

x = BatchNormalization()(x)

x = Dense(60*n60Feat, activation='relu')(x)
print(x.shape)
for i in range(2):
    x = Dense((60/(1))*n60Feat, activation='relu')(x)
    print(x.shape)

# add info to 1d output
y = BatchNormalization()(OneDInput)
y = Dense(n1dFeat, activation='relu')(y)

commonLayer = Concatenate(axis=1)([x, y])
commonLayerSize = 60*n60Targ+n1dFeat
y = Dense(commonLayerSize, activation='relu')(commonLayer)
print('y',y.shape)
for i in range(2):
    y = Dense(int(commonLayerSize / (2*(i+1))), activation='relu')(y)
    print('y',y.shape)
y = Dense(n1dTarg, activation='relu', name='1d')(y)
print('y',y.shape)
output1d =y

x = Dense(commonLayerSize, activation='relu')(commonLayer)
for i in range(0):
    x = Dense(commonLayerSize, activation='relu')(x)
x = Dense(60*n60Targ, activation='linear')(x)
print(x.shape)
#output2d = Reshape((60, n60Targ),name='2d')(x)
output2d = x
print(output2d.shape)


model2 = Model(inputs=[TwoDInput, OneDInput], outputs=output2d)
model2.compile(optimizer='adam', loss=squared_error_sum, metrics=[RSquaredMetric((60*n60Targ))])

hist = model2.fit([X2d, X1d], np.reshape(y2d, (y2d.shape[0],-1)), epochs=15, batch_size=256, validation_data=([X2d_val, X1d_val],np.reshape(y2d_val, (y2d_val.shape[0],-1))))

In [ ]:
y2d_pred = model2.predict([X2d_val, X1d_val])

y2d_pred0 = np.reshape(y2d_pred, (y2d_pred.shape[0],-1))
y2d_val0 = np.reshape(y2d_val, (y2d_val.shape[0],-1))
r2_scores = []
f = np.reshape(np.reshape(np.array(targets60), (n60Targ,60)).transpose(), (1,-1))
for i in range(y2d_val0.shape[1]):
    r2 = r2_score(y2d_val0[:, i], y2d_pred0[:, i])
    print(f[0][i], r2)
    r2_scores.append(r2)

In [ ]:
min(r2_scores),np.mean(r2_scores)

In [ ]:
m = RSquaredMetric2D((60,n60Targ))
m.update_state(y2d_val, y2d_pred)
m.result()

In [ ]:
m.total_sum_squares

# flatten input fully connected

In [ ]:
def squared_error_sum(y_true, y_pred):
    squared_error = tf.square(y_true - y_pred)
    return tf.reduce_sum(squared_error)

In [ ]:
tf.random.set_seed(42)

#OneDInput = Input(shape=(n1dFeat,))
numF = len(allF)
TwoDInput = Input(shape=(numF))

#y = BatchNormalization()(OneDInput)
#y = Dense(n1dFeat, activation='relu')(y)

x = BatchNormalization()(TwoDInput)
x = Dense(numF, activation='relu')(x)

#x = Concatenate(axis=1)([x, y])
commonSize = numF# + n1dFeat
print(x.shape)
for i in range(2):
    x = Dense((i+1)*commonSize, activation='relu')(x)
    print(x.shape)
for i in range(2):
    x = Dense(1/(i+1)*x.shape[1], activation='relu')(x)
    print(x.shape)
#for i in range(4):
#    x = Dense(commonSize, activation='relu')(x)
#    print(x.shape)
#
#y = Dense(, activation='linear',name='1d')(x)
x = Dense(60*n60Targ+n1dTarg, activation='linear',name='2d')(x)

print(x.shape)
output2d =x
#output1d =y

#model = Model(inputs=[TwoDInput, OneDInput], outputs=[output2d])#, output1d])
model = Model(inputs=[TwoDInput], outputs=[output2d])#, output1d])
model.compile(optimizer='adam', loss=squared_error_sum, metrics=[RSquaredMetric()])
model.summary()

#hist = model.fit([np.reshape(X2d,(X2d.shape[0],-1)), X1d], [np.reshape(y2d,(y2d.shape[0],-1)), y1d], epochs=10, batch_size=32, validation_data=([np.reshape(X2d_val,(X2d_val.shape[0],-1)), X1d_val],[np.reshape(y2d_val,(y2d_val.shape[0],-1)), y1d_val]))
#hist = model.fit([np.reshape(X2d,(X2d.shape[0],-1)), X1d], [np.reshape(y2dn,(y2dn.shape[0],-1))], epochs=15, batch_size=256, validation_data=([np.reshape(X2d_val,(X2d_val.shape[0],-1)), X1d_val],[np.reshape(y2dn_val,(y2dn_val.shape[0],-1))]))
hist = model.fit(X, yn, epochs=15, batch_size=512, validation_data=(X_val,yn_val))

In [ ]:
#loss: 46364.5664 - r_squared: 0.4725 - val_loss: 52628.4141 - val_r_squared: -2595754.2500
y2d_pred = model.predict(X_val)
y2d_pred = y2d_pred *std + mean

r2_scores = []
for i in range(y2d_pred.shape[1]):
    r2 = r2_score(y_val[:, i], y2d_pred[:, i])
    print(allTargets[i], r2)
    r2_scores.append(r2)

In [ ]:
np.mean(r2_scores)
#-1886738.6594913297
#-2311049.176559818
#-7024.681155863691  with features & normalized data

In [ ]:
y2d_pred = model.predict(X)
y2d_pred = y2d_pred *std + mean

r2_scores = []
for i in range(y2d_pred.shape[1]):
    r2 = r2_score(y[:, i], y2d_pred[:, i])
    print(allTargets[i], r2)
    r2_scores.append(r2)

In [ ]:
# batch size 1024
#loss: 5.6184e-10 - r_squared: -149219557376.0000 - val_loss: 9.7947e-10 - val_r_squared: -4477976313856.0000

# error = sum, bigger layer in middle helps
#loss: 5.1245e-05 - r_squared: 0.2087 - val_loss: 5.1066e-05 - val_r_squared: 0.2107
#bigger layer in between
#loss: 1.7005e-05 - r_squared: -1751025319936.0000 - val_loss: 1.9207e-05 - val_r_squared: -13320339456.000